<a href="https://colab.research.google.com/github/shannonshih/Tibame_GAD245-Practice-HW_0319/blob/main/Scikit_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

安裝對應的SDK模組

In [1]:
!pip install google-api-core --quiet
!pip install google-cloud-pubsub google-cloud-bigquery-storage --quiet
!pip install apache-beam[gcp,dataframe] --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.1/212.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.5/274.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.1/526.1 kB 27.3 MB/s 

授予訪問權限

In [2]:
from google.colab import auth
auth.authenticate_user()

導入模組

In [3]:
import pickle
from sklearn import linear_model
from typing import Tuple

import numpy as np
import apache_beam as beam

from apache_beam.ml.inference.sklearn_inference import ModelFileType
from apache_beam.ml.inference.sklearn_inference import SklearnModelHandlerNumpy
from apache_beam.ml.inference.base import KeyedModelHandler
from apache_beam.ml.inference.base import PredictionResult
from apache_beam.ml.inference.base import RunInference
from apache_beam.options.pipeline_options import PipelineOptions

定義常數 設定專案ID及值區ID

In [5]:
import os
# 定義常數（Constants）
project = "tibame-gad245-16-0319"
bucket = "tibame-gad245-16-0319"

# 設定專案 ID。
os.environ['GOOGLE_CLOUD_PROJECT'] = project

建立線性回歸模型

In [6]:
# 準備訓練 sklearn 模型的輸入資料（5 倍表）。
x = np.arange(0, 100, dtype=np.float32).reshape(-1, 1)
y = (x * 5).reshape(-1, 1)

def train_and_save_model(x, y, model_file_name):
  """訓練線性回歸模型並儲存至檔案。"""
  regression = linear_model.LinearRegression()
  regression.fit(x, y)

  with open(model_file_name, 'wb') as f:
      pickle.dump(regression, f)

# 訓練並儲存 5 倍表模型。
five_times_model_filename = 'sklearn_5x_model.pkl'
train_and_save_model(x, y, five_times_model_filename)

# 訓練並儲存 10 倍表模型。
ten_times_model_filename = 'sklearn_10x_model.pkl'
train_and_save_model(x, y, ten_times_model_filename)
y = (x * 10).reshape(-1, 1)
train_and_save_model(x, y, 'sklearn_10x_model.pkl')

讀取BigQuery資料進行預測

In [7]:
sklearn_model_handler = SklearnModelHandlerNumpy(model_uri=five_times_model_filename)

# 設定 Dataflow 管道選項，並指定暫存位置。
pipeline_options = PipelineOptions().from_dictionary(
                                      {'temp_location': f'gs://{bucket}/tmp'})

# 定義 BigQuery 資料表規格。
table_name = 'maths_problems_1'
table_spec = f'{project}:maths.{table_name}'

with beam.Pipeline(options=pipeline_options) as p:
  (
      p
      | "從 BigQuery 讀取資料" >> beam.io.ReadFromBigQuery(table=table_spec)
      | "提取輸入值" >> beam.Map(lambda x: [x['value']])
      | "執行 Sklearn 推論" >> RunInference(model_handler=sklearn_model_handler)
      | beam.Map(print)
  )

PredictionResult(example=[1013.0], inference=array([5065.]), model_id='sklearn_5x_model.pkl')
PredictionResult(example=[108.0], inference=array([540.]), model_id='sklearn_5x_model.pkl')
PredictionResult(example=[105.0], inference=array([525.]), model_id='sklearn_5x_model.pkl')
PredictionResult(example=[1000.0], inference=array([5000.]), model_id='sklearn_5x_model.pkl')


讀取鍵值資料預測

In [8]:
sklearn_model_handler = SklearnModelHandlerNumpy(model_uri=five_times_model_filename)
keyed_sklearn_model_handler = KeyedModelHandler(sklearn_model_handler)

# 設定 Dataflow 管道選項，並指定暫存位置。
pipeline_options = PipelineOptions().from_dictionary(
                                      {'temp_location': f'gs://{bucket}/tmp'})

with beam.Pipeline(options=pipeline_options) as p:
  (
  p
  | "從 BigQuery 讀取資料" >> beam.io.ReadFromBigQuery(table=table_spec)
  | "提取輸入值" >> beam.Map(lambda x: (x['key'], [x['value']]))
  | "執行 Sklearn 推論" >> RunInference(model_handler=keyed_sklearn_model_handler)
  | beam.Map(print)
  )

('fourth_question', PredictionResult(example=[1013.0], inference=array([5065.]), model_id='sklearn_5x_model.pkl'))
('second_question', PredictionResult(example=[108.0], inference=array([540.]), model_id='sklearn_5x_model.pkl'))
('first_question', PredictionResult(example=[105.0], inference=array([525.]), model_id='sklearn_5x_model.pkl'))
('third_question', PredictionResult(example=[1000.0], inference=array([5000.]), model_id='sklearn_5x_model.pkl'))


多模型管道預測

In [9]:
from typing import Tuple

def format_output(run_inference_output) -> str:
  """從 RunInference 的輸出提取 scikit-learn 預測結果。"""
  key, prediction_result = run_inference_output
  example = prediction_result.example[0]
  prediction = prediction_result.inference[0]
  return f"鍵值 = {key}, 範例 = {example} -> 預測結果 {prediction}"

five_times_model_handler = KeyedModelHandler(
    SklearnModelHandlerNumpy(model_uri=five_times_model_filename))
ten_times_model_handler = KeyedModelHandler(
    SklearnModelHandlerNumpy(model_uri=ten_times_model_filename))

# 設定 Dataflow 管道選項，並指定暫存位置。
pipeline_options = PipelineOptions().from_dictionary(
                                      {'temp_location': f'gs://{bucket}/tmp'})

with beam.Pipeline(options=pipeline_options) as p:
  inputs = (p
    | "從 BigQuery 讀取資料" >> beam.io.ReadFromBigQuery(table=table_spec))

  five_times = (inputs
    | "提取 5 倍運算" >> beam.Map(lambda x: ('{} {}'.format(x['key'], '* 5'), [x['value']]))
    | "執行 5 倍推論" >> RunInference(model_handler = five_times_model_handler))

  ten_times = (inputs
    | "提取 10 倍運算" >> beam.Map(lambda x: ('{} {}'.format(x['key'], '* 10'), [x['value']]))
    | "執行 10 倍推論" >> RunInference(model_handler = ten_times_model_handler))

  _ = ((five_times, ten_times)
    | "合併結果" >> beam.Flatten()
    | "格式化輸出" >> beam.Map(format_output)
    | "輸出結果" >> beam.Map(print))

鍵值 = fourth_question * 10, 範例 = 1013.0 -> 預測結果 10130.0
鍵值 = second_question * 10, 範例 = 108.0 -> 預測結果 1080.0
鍵值 = first_question * 10, 範例 = 105.0 -> 預測結果 1050.0
鍵值 = third_question * 10, 範例 = 1000.0 -> 預測結果 10000.0
鍵值 = fourth_question * 5, 範例 = 1013.0 -> 預測結果 5065.0
鍵值 = second_question * 5, 範例 = 108.0 -> 預測結果 540.0
鍵值 = first_question * 5, 範例 = 105.0 -> 預測結果 525.0
鍵值 = third_question * 5, 範例 = 1000.0 -> 預測結果 5000.0
